In [6]:
import json
import pandas as pd
import plotly.express as px

import numpy as np

import pandas as pd
import plotly.graph_objs as go
import plotly.express as px
from flask_babel import gettext
from plotly.subplots import make_subplots
from scipy import stats
import numpy as np
import math
df_prov_tot = pd.read_csv('static/csv/be-covid-provinces-all.csv')
from datetime import datetime, date
df_prov_tot.dropna(inplace=True) 
df_prov_tot['TOTAL_IN_PER_100K'] = df_prov_tot['TOTAL_IN'] / df_prov_tot['POP'] * 100000
pop = {'Brussels': 1218255, 'Flanders': 6629143, 'Wallonia': 3645243}


# Hospital Data

In [7]:
def hospi_prov(prov=None):
    df_hospi = df_prov_tot

    if prov != 'All':
        df_hospi = df_hospi[df_hospi['PROVINCE'] == prov]

    df = df_hospi.groupby(['DATE']).agg({'TOTAL_IN': 'sum', 'NEW_OUT': 'sum', 'NEW_IN': 'sum', 'TOTAL_IN_ICU': 'sum'})

    x2 = df.index
    y2 = df.NEW_IN

    df.index = pd.to_datetime(df.index)
    dw = df.index.dayofweek
    colors_map = {0: '#fccde5', 1: '#8dd3c7', 2: '#b3de69', 3: '#bebada', 4: '#fb8072', 5: '#fdb462', 6: '#80b1d3',
                  7: 'lightgrey'}
    colors = [colors_map[i] for i in dw]

    
    fig = go.Figure(data=[
        go.Bar(x=x2, y=y2, name="bars", 
               legendgroup='group2', showlegend=True, marker_color=colors),
        go.Scatter(x=x2, y=y2.rolling(7,center=True).mean(), 
                   showlegend=True, name=gettext('avg'),legendgroup='avg 7 days', marker_color="red")
    ])
    start = '2021-01-01' 
    fig.update_layout(yaxis_range=[0,max(y2[x2 >=start])])
    fig.update_layout(xaxis_range=[start, datetime.today().strftime('%Y-%m-%d')])
    fig.update_layout(template="plotly_white", height=300, margin=dict(l=0, r=0, t=30, b=0),title="Admisssions "+prov)

    return fig


def hospi_new_in_per_provinces():
    #prov = ['All','Liège', 'Brussels','Antwerpen','Namur', 'Luxembourg', 'Hainaut' , 'BrabantWallon', 'VlaamsBrabant', 'OostVlaanderen',
    #    'WestVlaanderen', 'Limburg',]
    prov = ['All']
    n_cols = 2
    n_rows = math.ceil(1.0 * len(prov)/ n_cols)

    idx = 0
    for prov_name in prov:
            fig = hospi_prov(prov=prov_name)
            
            fig.show()



hospi_new_in_per_provinces()

In [8]:
def bar_hospitalization_tot():
    """
    bar plot hospitalization
    """
    df = df_prov_tot.groupby(['DATE']).agg({'TOTAL_IN': 'sum', 'NEW_OUT': 'sum', 'NEW_IN': 'sum', 'TOTAL_IN_ICU': 'sum'})

    totin_bar = go.Scatter(x=df.index, y=df.TOTAL_IN, name='#Tot',stackgroup='two',line=dict(width=0.5),mode='lines', groupnorm="")

    fig_hospi = make_subplots(specs=[[{"secondary_y": True, }]], shared_yaxes='all', shared_xaxes='all')

    fig_hospi.add_trace(totin_bar, secondary_y=False)

    xvals = df.index[7:]
    yvals = 100 * (df.TOTAL_IN.values[7:] - df.TOTAL_IN.values[:-7]) / df.TOTAL_IN.values[:-7]

    fig_hospi.add_trace(go.Scatter(x=xvals[10:], y=yvals[10:], name='Week%',marker_color = 'pink'), secondary_y=True)

    fig_hospi.update_layout(template="plotly_white", height=500, margin=dict(l=0, r=0, t=30, b=0),
                            title="Total Hospitalizations")

    fig_hospi.update_layout(xaxis_title='Day',
                            yaxis_title='Number of / Day')

    fig_hospi.update_layout(
        hovermode='x unified',
        updatemenus=[
            dict(
                type="buttons",
                direction="left",
                buttons=list([
                    dict(
                        args=[{"yaxis.type": "linear"}],
                        label="LINEAR",
                        method="relayout"
                    ),
                    dict(
                        args=[{"yaxis.type": "log"}],
                        label="LOG",
                        method="relayout"
                    )
                ]),
            ),
        ])

    return fig_hospi

bar_hospitalization_tot()

In [9]:
def bar_icu_tot():

    df = df_prov_tot.groupby(['DATE']).agg({'TOTAL_IN': 'sum', 'NEW_OUT': 'sum', 'NEW_IN': 'sum', 'TOTAL_IN_ICU': 'sum'})

    totin_bar = go.Scatter(x=df.index, y=df.TOTAL_IN_ICU, name='#Tot',stackgroup='two',line=dict(width=0.5),mode='lines', groupnorm="")

    fig_hospi = make_subplots(specs=[[{"secondary_y": True, }]], shared_yaxes='all', shared_xaxes='all')

    fig_hospi.add_trace(totin_bar, secondary_y=False)

    xvals = df.index[7:]
    yvals = 100 * (df.TOTAL_IN_ICU.values[7:] - df.TOTAL_IN_ICU.values[:-7]) / df.TOTAL_IN_ICU.values[:-7]

    fig_hospi.add_trace(go.Scatter(x=xvals[10:], y=yvals[10:], name='Week%',marker_color = 'pink'), secondary_y=True)

    fig_hospi.update_layout(template="plotly_white", height=500, margin=dict(l=0, r=0, t=30, b=0),
                            title="ICU")

    fig_hospi.update_layout(xaxis_title='Day',
                            yaxis_title='Number of / Day')

    fig_hospi.update_layout(
        hovermode='x unified',
        updatemenus=[
            dict(
                type="buttons",
                direction="left",
                buttons=list([
                    dict(
                        args=[{"yaxis.type": "linear"}],
                        label="LINEAR",
                        method="relayout"
                    ),
                    dict(
                        args=[{"yaxis.type": "log"}],
                        label="LOG",
                        method="relayout"
                    )
                ]),
            ),
        ])

    return fig_hospi

bar_icu_tot()

# Hospital Data Per Region

In [10]:
def hospi_region_per100k(column,avg=False):
    barmode = 'group'  # group
    # bar plot with bars per age groups
    bars = []
    regions = sorted(df_prov_tot.REGION.unique())

    idx = pd.date_range(df_prov_tot.DATE.min(), df_prov_tot.DATE.max())

    colors = px.colors.qualitative.Plotly
    i = 0

    for r in regions:
        c = colors[i]
        df_r = df_prov_tot.loc[df_prov_tot['REGION'] == r]
        df_r = df_r.groupby(['DATE']).agg({column: 'sum'})
        df_r.index = pd.DatetimeIndex(df_r.index)
        df_r = df_r.reindex(idx, fill_value=0)

        if avg:
            plot = go.Scatter(x=df_r.index, y=(100000 * df_r[column] / pop[r]).rolling(7,center=True).mean(),
                              name=r+" avg",mode="lines",marker_color=c)
            bars.append(plot)
            plot = go.Bar(x=df_r.index, y=(100000 * df_r[column] / pop[r]),
                              name=r,marker_color=c)
            bars.append(plot)
        else:
            plot = go.Scatter(x=df_r.index, y=(100000 * df_r[column] / pop[r]),
                              name=r,marker_color=c)
            bars.append(plot)
        i += 1

    fig = go.Figure(data=bars )
    fig.update_layout(template="plotly_white", height=500, barmode=barmode, margin=dict(l=0, r=0, t=30, b=0), )

    fig.update_layout(template="plotly_white", title=column+" per 100K inhabitants" + ("(avg 7 days)" if avg  else ""))
    fig.update_layout(
        hovermode='x unified',
        updatemenus=[
            dict(
                type="buttons",
                direction="left",
                buttons=list([
                    dict(
                        args=[{"yaxis.type": "linear"}],
                        label="LINEAR",
                        method="relayout"
                    ),
                    dict(
                        args=[{"yaxis.type": "log"}],
                        label="LOG",
                        method="relayout"
                    )
                ]),
            ),
        ])

    return fig

hospi_region_per100k("NEW_IN",avg=True).show()
hospi_region_per100k("TOTAL_IN",avg=True).show()
hospi_region_per100k("TOTAL_IN_ICU",avg=True).show()

# Hospital Data Per province

In [11]:
def total_new_in_provinces():
    fig = px.area(df_prov_tot, x="DATE", y="NEW_IN", color="PROVINCE",title="Admissions")
    fig.update_layout(template="plotly_white")
    fig.update_layout(
    updatemenus=[
        dict(
            type="buttons",
            direction="up",
            buttons=list([
                dict(
                    args=[{"groupnorm": ""}],
                    label="absolute",
                    method="restyle"
                ),
                dict(
                    args=[{"groupnorm": "percent"}],
                    label="percent",
                    method="restyle"
                )
            ]),
        ),
    ])
    return fig
total_new_in_provinces()

In [12]:
def hospi_provinces():
    fig = px.area(df_prov_tot, x="DATE", y="TOTAL_IN", color="PROVINCE",title="Total number of hospitalized patients")
    fig.update_layout(template="plotly_white")
    fig.update_layout(
    updatemenus=[
        dict(
            type="buttons",
            direction="up",
            buttons=list([
                dict(
                    args=[{"groupnorm": ""}],
                    label="absolute",
                    method="restyle"
                ),
                dict(
                    args=[{"groupnorm": "percent"}],
                    label="percent",
                    method="restyle"
                )
            ]),
        ),
    ])
    return fig
hospi_provinces()

In [13]:
def total_icu_provinces():
    fig = px.area(df_prov_tot, x="DATE", y="TOTAL_IN_ICU", color="PROVINCE",title="Total number of hospitalized patients in ICU at the moment of reporting")
    fig.update_layout(template="plotly_white")
    fig.update_layout(
    updatemenus=[
        dict(
            type="buttons",
            direction="up",
            buttons=list([
                dict(
                    args=[{"groupnorm": ""}],
                    label="absolute",
                    method="restyle"
                ),
                dict(
                    args=[{"groupnorm": "percent"}],
                    label="percent",
                    method="restyle"
                )
            ]),
        ),
    ])
    return fig
total_icu_provinces()